In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
from pathlib import Path

# Define PROJECT_PATH
PROJECT_PATH = Path("/project/greencenter/Toprak_lab/shared/TEM1_Combinatorial_Mutagenesis/src/Epistasis")

# Load datasets
df = pd.read_parquet(PROJECT_PATH / "data/processed/Epistasis_Combined.parquet")

In [3]:
df

,Genotype,Epistatic Term,Epistatic Order,Fitness,Error,Biochemical Definition,Error_Bioch,Ensemble Averaging,Error_EA,Drug,...,Fitness_predicted for order 10,Epistasis_LG for order 11,Error_LG for order 11,Fitness_predicted for order 11,Epistasis_LG for order 12,Error_LG for order 12,Fitness_predicted for order 12,Epistasis_LG for order 13,Error_LG for order 13,Fitness_predicted for order 13
0,LQMERMAGERTRN,0000000000000,0,3.206299,0.043505,3.212891,0.043610,3.183594,0.000000,AZT,...,3.195312,3.216797,0.044250,3.216797,3.208984,0.043610,3.208984,3.212891,0.043610,3.212891
1,LQMERMAGERTRD,0000000000001,1,3.138404,0.054579,-0.068359,0.069824,0.000258,0.000000,AZT,...,3.152344,-0.081543,0.071289,3.134766,-0.062744,0.069885,3.146484,-0.068359,0.069824,3.144531
2,LQMERMAGERTLN,0000000000010,1,3.162856,0.047664,-0.044922,0.064514,-0.003452,0.000000,AZT,...,3.187500,-0.047821,0.065796,3.169922,-0.043152,0.064575,3.166016,-0.044922,0.064514,3.167969
3,LQMERMAGERTLD,0000000000011,2,3.146823,0.086617,0.052734,0.120972,0.002346,0.000000,AZT,...,3.136719,0.068420,0.122742,3.156250,0.049194,0.121155,3.152344,0.052734,0.120972,3.152344
4,LQMERMAGERTQN,0000000000020,1,3.170136,0.132060,-0.037109,0.139038,-0.000529,0.000000,AZT,...,3.181641,-0.049255,0.138794,3.167969,-0.030426,0.139038,3.177734,-0.037109,0.139038,3.175781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774139,PKVKNTTSKCMRD,1121311112101,12,1.985129,0.731761,61.906250,25.093750,17.421875,15.273438,AMP,...,1.988281,0.000000,0.000000,1.990234,17.421875,15.273438,1.828125,61.906250,25.093750,1.953125
774140,PKVKNTTSKCMLN,1121311112110,12,2.127111,0.337499,55.187500,25.265625,20.906250,18.328125,AMP,...,1.832031,0.000000,0.000000,2.218750,20.906250,18.328125,2.195312,55.187500,25.265625,2.093750
774141,PKVKNTTSKCMLD,1121311112111,13,2.640538,0.212470,-68.625000,36.656250,-68.625000,36.656250,AMP,...,2.460938,0.000000,0.000000,2.429688,0.000000,0.000000,2.738281,-68.625000,36.656250,2.558594
774142,PKVKNTTSKCMQN,1121311112120,12,2.279293,0.096776,15.242188,25.906250,-17.218750,18.593750,AMP,...,2.460938,0.000000,0.000000,2.421875,-17.218750,18.593750,2.162109,15.242188,25.906250,2.296875


In [9]:
unique_genotypes_azt = df[df["Drug"] == "AZT"]["Genotype"].nunique()
unique_genotypes_amp = df[df["Drug"] == "AMP"]["Genotype"].nunique()

print(f"Number of unique genotypes in AZT dataset: {unique_genotypes_azt}")
print(f"Number of unique genotypes in AMP dataset: {unique_genotypes_amp}")

Number of unique genotypes in AZT dataset: 55296
Number of unique genotypes in AMP dataset: 55296


In [4]:
# Load the encoded variants
clinical_variants = pd.read_csv(PROJECT_PATH / "figures/known_variants/encoded_variants.csv")
clinical_variants


,Variant,Mutations,Encoded_Sequence
0,TEM-2,Q39K,LKMERMAGERTRN
1,TEM-3,Q39K-E104K-G238S,LKMKRMASERTRN
2,TEM-5,R164S-A237T-E240K,LQMESMTGKRTRN
3,TEM-6,E104K-R164H,LQMKHMAGERTRN
4,TEM-7,Q39K-R164S,LKMESMAGERTRN
...,...,...,...
58,TEM-168,T265M,LQMERMAGERMRN
59,TEM-189,M69L-E240K,LQLERMAGKRTRN
60,TEM-191,E240K,LQMERMAGKRTRN
61,TEM-203,Q39K-E240K,LKMERMAGKRTRN


In [5]:
df["Genotype"].isin(clinical_variants["Encoded_Sequence"]).sum()

854

In [ ]:
# Filter the DataFrame for specific conditions
common_genotypes = df["Genotype"].unique().intersection(set(clinical_variants["Encoded_Sequence"]))
print(f"Clinical sequences in our experiment: {len(common_genotypes)}")

# Create merged dataset with fitness values for matches
clinical_matches = pd.DataFrame(columns=["Variant", "Genotype", "Fitness_AMP", "Fitness_AZT"])

for genotype in common_genotypes:
    # Get the variant name
    variant_name = clinical_variants.loc[clinical_variants["Encoded_Sequence"] == genotype, "Variant"].iloc[0]
    
    # Get fitness values
    amp_fitness = matches_in_amp.loc[matches_in_amp["Genotype"] == genotype, "Fitness"].iloc[0]
    azt_fitness = matches_in_azt.loc[matches_in_azt["Genotype"] == genotype, "Fitness"].iloc[0]
    
    # Create a new row and append it
    new_row = pd.DataFrame({
        "Variant": [variant_name],
        "Genotype": [genotype],
        "Fitness_AMP": [amp_fitness],
        "Fitness_AZT": [azt_fitness]
    })
    clinical_matches = pd.concat([clinical_matches, new_row], ignore_index=True)

print("\nMatched clinical isolates:")
print(clinical_matches)


AttributeError: 'set' object has no attribute 'unique'

In [ ]:

def plot_fitness_AMP_vs_AZT(amp_fitness, azt_fitness, highlight_df=None):
    fig = plt.figure(figsize=(6, 6))
    gs = fig.add_gridspec(2, 2, width_ratios=[3, 1], height_ratios=[1, 3],
                          left=0.1, right=0.9, bottom=0.1, top=0.9,
                          wspace=0.05, hspace=0.05)
    
    ax = fig.add_subplot(gs[1, 0])
    amp_bins = np.linspace(-0.05, 1.15, 50)
    azt_bins = np.linspace(-1, 6.5, 50)
    hist, xedges, yedges, img = ax.hist2d(
        amp_fitness, azt_fitness, bins=[amp_bins, azt_bins],
        cmap='BuPu', norm=LogNorm()
    )
    
    # Add extra variants with dummy fitness values
    extra_variants = {
        'c1.1': {'sequence': 'LQMKNTTAGKRTRN', 'amp_fitness': 0.75, 'azt_fitness': 2.8},
        'c1.2': {'sequence': 'PQMKNMAGKRMRN', 'amp_fitness': 0.82, 'azt_fitness': 3.2},
        'c1.3': {'sequence': 'LKMKSMAGKRMRN', 'amp_fitness': 0.9, 'azt_fitness': 3.5},
        'c2.1': {'sequence': 'LQMKNMAGKRMRN', 'amp_fitness': 0.85, 'azt_fitness': 4.0},
        'c2.2': {'sequence': 'LKMNMAGKRTRN', 'amp_fitness': 0.78, 'azt_fitness': 4.3},
        'c2.3': {'sequence': 'LKMNTAGKRTRN', 'amp_fitness': 0.95, 'azt_fitness': 4.7},
        'c3.1': {'sequence': 'PQMKNTAGKRTRN', 'amp_fitness': 0.88, 'azt_fitness': 5.0}
    }
    
    # Add yellow asterisks for clinical isolates if we have data
    # Plot clinical isolates FIRST so X markers appear on top
    if highlight_df is not None and not highlight_df.empty:
        print(f"Plotting {len(highlight_df)} clinical isolates")
        print("Clinical isolates data:")
        print(highlight_df[["Variant", "Fitness_AMP", "Fitness_AZT"]].sort_values(by="Fitness_AZT", ascending=False))
        
        ax.plot(
            highlight_df["Fitness_AMP"],
            highlight_df["Fitness_AZT"],
            marker='*',
            linestyle='none',
            markerfacecolor='yellow',
            markeredgecolor='black',
            markersize=12,
            markeredgewidth=0.5,
            label='Clinical Isolates'
        )
    
    # Plot extra variants with green stars
    extra_amp_fitness = [extra_variants[v]['amp_fitness'] for v in extra_variants]
    extra_azt_fitness = [extra_variants[v]['azt_fitness'] for v in extra_variants]
    extra_labels = list(extra_variants.keys())
    
    ax.plot(
        extra_amp_fitness,
        extra_azt_fitness,
        marker='*',
        linestyle='none',
        markerfacecolor='green',
        markeredgecolor='black',
        markersize=12,
        markeredgewidth=0.5,
        label='New Variants'
    )
    
    # Add annotations for the extra variants
    for i, variant in enumerate(extra_labels):
        ax.annotate(
            variant,
            (extra_amp_fitness[i], extra_azt_fitness[i]),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=6,
            color='darkgreen'
        )
    
    # Plot X markers AFTER clinical isolates so they appear on top
    ax.plot(0, 0, 'x', color='blue', markersize=10, markeredgewidth=2, label='TEM-1$^{dead}$')
    ax.plot(1, 1, 'x', color='black', markersize=10, markeredgewidth=2, label='TEM-1$^{WT}$')
    
    ax.legend(frameon=False, fontsize=8, loc='upper left')

    cbar_ax = fig.add_axes([0.9, 0.37, 0.03, 0.3])
    fig.colorbar(img, cax=cbar_ax)

    ax_top = fig.add_subplot(gs[0, 0], sharex=ax)
    ax_top.hist(amp_fitness, bins=amp_bins, color='gray', alpha=0.95, edgecolor='black', linewidth=0.5)
    ax_top.set_ylim(0.1, ax_top.get_ylim()[1])
    ax_top.tick_params(labelleft=False, labelbottom=False)
    ax_top.set_yticks([])

    ax_right = fig.add_subplot(gs[1, 1], sharey=ax)
    ax_right.hist(azt_fitness, bins=azt_bins, orientation='horizontal', color='hotpink', alpha=0.95, edgecolor='black', linewidth=0.5)
    ax_right.set_xlim(0.1, ax_right.get_xlim()[1])
    ax_right.tick_params(labelleft=False, labelbottom=False)
    ax_right.set_xticks([])

    ax.set_xlabel('Fitness$_{global}$ in Ampicillin', fontsize=12, labelpad=3, fontname='Arial')
    ax.set_ylabel('Fitness$_{global}$ in Aztreonam', fontsize=12, labelpad=1, fontname='Arial')

    amp_ticks = [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
    azt_ticks = [0, 1, 2, 3, 4, 5, 6, 7]
    ax.set_xticks(amp_ticks)
    ax.set_xticklabels(amp_ticks, rotation=0, fontsize=8, fontname='Arial')
    ax.set_yticks(azt_ticks)
    ax.set_yticklabels(azt_ticks, rotation=0, fontsize=8, fontname='Arial')
    
    ax.plot([-0.1, 1.2], [-0.1, 1.2], 'k--', lw=1, alpha=0.95)
    ax.set_xlim(-0.05, 1.1)
    ax.set_ylim(-0.5, 8)

    # Get maximum counts from both histograms for setting common y-limit
    top_counts, _ = np.histogram(amp_fitness, bins=amp_bins)
    right_counts, _ = np.histogram(azt_fitness, bins=azt_bins)
    max_count = max(np.max(top_counts), np.max(right_counts))
    
    # Update y-limit for top histogram and x-limit for right histogram
    ax_top.set_ylim(0.1, max_count * 1.01)
    ax_right.set_xlim(0.1, max_count * 1.01)

    ax_top.spines['top'].set_visible(False)
    ax_top.spines['right'].set_visible(False)
    ax_top.spines['left'].set_visible(False)
    ax_right.spines['top'].set_visible(False)
    ax_right.spines['right'].set_visible(False)
    ax_right.spines['bottom'].set_visible(False)
    
    plt.savefig(PROJECT_PATH / "Fitness_AMP_vs_AZT_with_all_variants.png", 
                dpi=1200, bbox_inches='tight')
    plt.show()

# Run the analysis
def analyze_clinical_isolates(clinical_matches):
    print(f"\nAnalysis of {len(clinical_matches)} clinical isolates:")
    
    # Calculate statistics
    amp_mean = clinical_matches["Fitness_AMP"].mean()
    azt_mean = clinical_matches["Fitness_AZT"].mean()
    amp_std = clinical_matches["Fitness_AMP"].std()
    azt_std = clinical_matches["Fitness_AZT"].std()
    
    print(f"Mean AMP fitness: {amp_mean:.3f} ± {amp_std:.3f}")
    print(f"Mean AZT fitness: {azt_mean:.3f} ± {azt_std:.3f}")
    
    # Find isolates with high/low fitness in both drugs
    dual_resistance = clinical_matches[(clinical_matches["Fitness_AMP"] > amp_mean) & 
                                      (clinical_matches["Fitness_AZT"] > azt_mean)]
    dual_sensitivity = clinical_matches[(clinical_matches["Fitness_AMP"] < amp_mean) & 
                                       (clinical_matches["Fitness_AZT"] < azt_mean)]
    
    print(f"Isolates with above-average fitness in both drugs: {len(dual_resistance)}")
    if len(dual_resistance) > 0:
        print("Top dual-resistant variants:")
        for i, row in dual_resistance.sort_values(by=['Fitness_AMP', 'Fitness_AZT'], ascending=False).head(5).iterrows():
            print(f"  {row['Variant']}: AMP={row['Fitness_AMP']:.3f}, AZT={row['Fitness_AZT']:.3f}")
    
    print(f"Isolates with below-average fitness in both drugs: {len(dual_sensitivity)}")

# Run the updated plot
plot_fitness_AMP_vs_AZT(
    amp_df["Fitness"],
    azt_df["Fitness"],
    clinical_matches
)

# Analyze the clinical isolates
analyze_clinical_isolates(clinical_matches)


NameError: name 'amp_df' is not defined